In [73]:
from array import array
import reprlib
import operator
import math
import functools
import itertools

class Vector:
    __match_args__ = ('x', 'y', 'z', 't')
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)  # used to limit the console logs with ...
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return len(self) == len(other) and all(a == b for a, b in zip(self, other))
        # 2nd iteration
        # if len(self) != len(other):
        #     return False
        # for a, b in zip(self, other):
        #     if a != b:
        #         return False
        # return True
        # 1st iteration
        # return tuple(self) == tuple(other) # creating two tuples, and what about large multidimensional vectors

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self)
            return cls(self._components[key])
        index = operator.index(key)
        return self._components[index]

    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components):
            return self._components[pos]
        msg = f'{cls.__name_!r} object has no attribute {name!r}'
        raise AttributeError(msg)

    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.__match_args__:
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)

    def __hash__(self):
        hashes = (hash(x) for x in self._components)
        # hashes = map(hash, self._components) # Python 2 would be slower here but Python 3 map is lazy it creates a generator
        return functools.reduce(operator.xor, hashes, 0)

    def angle(self, n):
        r = math.hypot(*self[n:])
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'): # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [2]:
Vector([3.1,4.2])

Vector([3.1, 4.2])

In [3]:
Vector((3,4,5))

Vector([3.0, 4.0, 5.0])

In [4]:
Vector(range(10))

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

In [6]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])
class FrenchDeck:
    ranks = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                       for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

In [8]:
v1 = Vector([3,4,5])
len(v1)

3

In [9]:
v1[0], v1[-1]

(3.0, 5.0)

In [18]:
v7 = Vector(range(7))
v7[1:4]

array('d', [1.0, 2.0, 3.0])

In [19]:
class MySeq:
    def __getitem__(self, index):
        return index

s = MySeq()
s[1]

1

In [20]:
s[1:4]

slice(1, 4, None)

In [21]:
s[1:4:2]

slice(1, 4, 2)

In [22]:
s[1:4:2, 9]

(slice(1, 4, 2), 9)

In [23]:
s[1:4:2, 7:9]

(slice(1, 4, 2), slice(7, 9, None))

In [24]:
slice

slice

In [25]:
dir(slice)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'indices',
 'start',
 'step',
 'stop']

In [28]:
slice(None, 10, 2).indices(5) # 'ABCDE' len == 5: 'ABCDE'[:10:2] same as 'ABCDE'[0:5:2]

(0, 5, 2)

In [29]:
slice(-3, None, None).indices(5) # 'ABCDE' len == 5: 'ABCDE'[-3:] same as 'ABCDE'[2:5:1]

(2, 5, 1)

In [35]:
v7 = Vector(range(7))
v7[-1]

6.0

In [36]:
v7[1:4]

Vector([1.0, 2.0, 3.0])

In [37]:
v7[-1:]

Vector([6.0])

In [40]:
v = Vector(range(10))
v.x

0.0

In [42]:
v.y, v.z, v.t

(1.0, 2.0, 3.0)

In [48]:
v = Vector(range(5))
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

In [49]:
v.x

0.0

In [50]:
v.x = 10
v.x # throw with how custom __setattr__
# we could implement __setitem__ and/or __setattr__ to make v.x = 10 work but it will remain immutable because we want to make it hashable

AttributeError: readonly attribute 'x'

In [51]:
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

In [52]:
2 * 3 * 4 * 5 #5!

120

In [58]:
import functools
functools.reduce(lambda a, b: a*b, range(1, 6))

120

In [57]:
n = 0
for i in range(1,6):
    n ^= i

n

1

In [59]:
functools.reduce(lambda a, b: a^b, range(1, 6))

1

In [60]:
functools.reduce(operator.xor, range(6))

1

In [62]:
zip(range(3), 'ABC')

In [63]:
list(zip(range(3), 'ABC'))

[(0, 'A'), (1, 'B'), (2, 'C')]

In [64]:
list(zip(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3]))

[(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2)]

In [66]:
from itertools import zip_longest
list(zip_longest(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3], fillvalue=-1))

[(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2), (-1, -1, 3.3)]

In [67]:
a = [(1,2,3),(4,5,6)]
list(zip(*a))

[(1, 4), (2, 5), (3, 6)]

In [68]:
b = [(1,2),
     (3,4),
     (5,6)]
list(zip(*b))

[(1, 3, 5), (2, 4, 6)]

In [74]:
format(Vector([1, 1]), 'h')

'<1.4142135623730951, 0.7853981633974483>'

In [75]:
format(Vector([1, 1]), '0.5fh')

'<1.41421, 0.78540>'

In [76]:
format(Vector([-1,-1,-1,-1]), 'h')

'<1.4142135623730951, 2.0943951023931957, 2.186276035465284, 3.9269908169872414>'

In [77]:
format(Vector([2,2,2,2]), '.3eh')

'<2.828e+00, 1.047e+00, 9.553e-01, 7.854e-01>'

In [78]:
format(Vector([0, 1, 0, 0]), '0.5fh')

'<1.00000, 1.57080, 0.00000, 0.00000>'